In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import pickle
import re

In [ ]:
match_urls = []

# create a base URL
base_url = 'https://fbref.com'
schedule_url = 'https://fbref.com/en/comps/20/2016-2017/schedule/2016-2017-Bundesliga-Scores-and-Fixtures'


# use request to get URL
schedule_page = requests.get(schedule_url)


# parse URL / content of the page; need to find the url to all matches
soup = BeautifulSoup(schedule_page.content, 'html.parser')
results = soup.find(id="sched_all")
matches = results.find_all("td", class_="center")

# go trough matches and access the url and combine it with the base url for a working url to the match
for match in matches:
    links = match.find_all("a")
    for link in links:
        url = base_url + link["href"]
        match_urls.append(url)

        
with open('MatchURLs', 'wb') as fp:
        pickle.dump(match_urls, fp)        
        
        
#with open('MatchURLs', 'rb') as fp:
#        match_urls = pickle.load(fp)


In [ ]:
Home = []
Away = []
home_pos = []
away_pos = []

for match_url in match_urls:
    time.sleep(12)
    text = []
    percentages = []


    # use request to get URL
    match_page = requests.get(match_url)

    # parse URL / content of the page
    soup = BeautifulSoup(match_page.content, 'html.parser')

    results = soup.find(id="content")
    scoreboxes = results.find_all("div", class_="scorebox")
    for scorebox in scoreboxes:
        ass = scorebox.find_all("a")
        for a in ass:
            text.append(a.text)

    Home.append(text[0])
    Away.append(text[4])
    
    results = soup.find(id="team_stats")
    tables = results.find_all('table')
    for table in tables:
        trs = table.find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            for td in tds:
                s = td.find_all('strong')
                percentages.append(str(s).replace('[<strong>', '').replace('%</strong>]', ''))

    home_pos.append(percentages[0])
    away_pos.append(percentages[1])
    


In [ ]:
possession_overview = pd.DataFrame(list(zip(Home, Away, home_pos, away_pos)),
                                           columns = ['Home', 'Away', 'Pos_Home', 'Pos_Away'])
possession_overview

In [ ]:
possession_overview = possession_overview[:306]
possession_overview.to_csv('Output/PossessionOverview.csv')

In [4]:
# match names

game_overview2= pd.read_csv('Output/GameOverview2.csv')
game_overview2.Home.unique()

array(['FC Bayern München', 'Borussia Dortmund',
       'Borussia Mönchengladbach', 'Hertha BSC', '1. FC Köln',
       'Hamburger SV', 'FC Augsburg', 'TSG 1899 Hoffenheim',
       'Eintracht Frankfurt', 'Bayer 04 Leverkusen', 'FC Schalke 04',
       '1. FSV Mainz 05', 'VfL Wolfsburg', 'FC Ingolstadt 04',
       'SV Darmstadt 98', 'Sport-Club Freiburg', 'RB Leipzig',
       'SV Werder Bremen'], dtype=object)

In [5]:
possession_overview = pd.read_csv('Output/PossessionOverview.csv')
possession_overview = possession_overview.iloc[:, 1:]
possession_overview.Home.unique()

array(['Bayern Munich', 'Dortmund', 'Hamburger SV', 'Eintracht Frankfurt',
       'Köln', 'Augsburg', 'Mönchengladbach', 'Hertha BSC', 'Hoffenheim',
       'Schalke 04', 'Bayer Leverkusen', 'Freiburg', 'Wolfsburg',
       'Ingolstadt 04', 'Darmstadt 98', 'RB Leipzig', 'Werder Bremen',
       'Mainz 05'], dtype=object)

In [6]:
possession_overview.replace(possession_overview.Home.unique(), ['FC Bayern München', 'Borussia Dortmund', 'Hamburger SV', 
                                                               'Eintracht Frankfurt', '1. FC Köln', 'FC Augsburg', 
                                                               'Borussia Mönchengladbach', 'Hertha BSC', 'TSG 1899 Hoffenheim',
                                                               'FC Schalke 04', 'Bayer 04 Leverkusen', 'Sport-Club Freiburg',
                                                               'VfL Wolfsburg', 'FC Ingolstadt 04', 'SV Darmstadt 98',
                                                               'RB Leipzig', 'SV Werder Bremen', '1. FSV Mainz 05'],
                            inplace= True)
possession_overview

,Home,Away,Pos_Home,Pos_Away
0,FC Bayern München,SV Werder Bremen,71,30
1,Borussia Dortmund,1. FSV Mainz 05,74,26
2,Hamburger SV,FC Ingolstadt 04,47,53
3,Eintracht Frankfurt,FC Schalke 04,44,56
4,1. FC Köln,SV Darmstadt 98,59,41
...,...,...,...,...
301,FC Bayern München,Sport-Club Freiburg,66,34
302,TSG 1899 Hoffenheim,FC Augsburg,76,24
303,1. FC Köln,1. FSV Mainz 05,46,54
304,Eintracht Frankfurt,RB Leipzig,42,58


In [8]:
new_overview = pd.merge(game_overview2, possession_overview, on = ['Home', 'Away'], how = 'inner')
new_overview = new_overview.iloc[:,1:]

new_overview.to_csv('Output/GameOverview3.csv')
new_overview

,MI_Date,Home,Away,Final_Score,Matchday,MI_Name,NameCorrect?,Pos_Name,Opta_Date,Opta_Name,Red_card,Retired,Direction_Home_HT1,Direction_Home_HT2,Direction_Away_HT1,Direction_Away_HT2,Pos_Home,Pos_Away
0,2016-08-26T20:30:00.000+02:00,FC Bayern München,SV Werder Bremen,6:0,1,DFL-MAT-00277X,True,DFL-MAT-00277X,2016-08-26T19:30:00,f24-22-2016-861390,False,False,Right to Left,Left to Right,Left to Right,Right to Left,71,30
1,2016-08-27T15:30:00.000+02:00,Borussia Dortmund,1. FSV Mainz 05,2:1,1,DFL-MAT-00277Y,True,DFL-MAT-00277Y,2016-08-27T14:30:00,f24-22-2016-861391,False,False,Left to Right,Right to Left,Right to Left,Left to Right,74,26
2,2016-08-27T18:30:00.000+02:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,2:1,1,DFL-MAT-00277Z,True,DFL-MAT-00277Z,2016-08-27T17:30:00,f24-22-2016-861392,False,True,Left to Right,Right to Left,Right to Left,Left to Right,51,50
3,2016-08-28T15:30:00.000+02:00,Hertha BSC,Sport-Club Freiburg,2:1,1,DFL-MAT-002780,True,DFL-MAT-002780,2016-08-28T14:30:00,f24-22-2016-861393,False,False,Right to Left,Left to Right,Left to Right,Right to Left,55,45
4,2016-08-27T15:30:00.000+02:00,1. FC Köln,SV Darmstadt 98,2:0,1,DFL-MAT-002781,True,DFL-MAT-002781,2016-08-27T14:30:00,f24-22-2016-861394,False,True,Right to Left,Left to Right,Left to Right,Right to Left,59,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2017-05-20T15:30:00.000+02:00,1. FC Köln,1. FSV Mainz 05,2:0,34,DFL-MAT-0027GA,True,DFL-MAT-0027GA,2017-05-20T14:30:00,f24-22-2016-861691,False,False,Right to Left,Left to Right,Left to Right,Right to Left,46,54
298,2017-05-20T15:30:00.000+02:00,Hamburger SV,VfL Wolfsburg,2:1,34,DFL-MAT-0027GB,True,DFL-MAT-0027GB,2017-05-20T14:30:00,f24-22-2016-861692,False,False,Right to Left,Left to Right,Left to Right,Right to Left,49,51
299,2017-05-20T15:30:00.000+02:00,FC Ingolstadt 04,FC Schalke 04,1:1,34,DFL-MAT-0027GC,True,DFL-MAT-0027GC,2017-05-20T14:30:00,f24-22-2016-861693,True,False,Left to Right,Right to Left,Right to Left,Left to Right,50,50
300,2017-05-20T15:30:00.000+02:00,TSG 1899 Hoffenheim,FC Augsburg,0:0,34,DFL-MAT-0027GD,True,DFL-MAT-0027GD,2017-05-20T14:30:00,f24-22-2016-861694,False,False,Right to Left,Left to Right,Left to Right,Right to Left,76,24
